In [ ]:
import streamlit as st
import requests
from elasticsearch import Elasticsearch

def check_server(client):
    if not client.ping():
        st.error("ElasticSearch server is not running!", icon="🚨")
        st.stop()

def display_results(client, query, search_type):
    check_server(client)
    
    search_request = {
        search_type: {"tags": query}
    }
    
    results = client.search(index="flickrphotos", query=search_request)
    
    st.subheader(f"Displaying Results for: {query}")
    
    cols = st.columns(2)
    col_heights = [0, 0]
    displayed_images = 0
    
    for hit in results["hits"]['hits']:
        if displayed_images >= 16:
            break
        image_data = hit["_source"]
        image_url = f"http://farm{image_data['flickr_farm']}.staticflickr.com/{image_data['flickr_server']}/{image_data['id']}_{image_data['flickr_secret']}.jpg"
        
        col_idx = 0 if col_heights[0] <= col_heights[1] else 1
        with st.spinner("Loading Image..."):
            response = requests.get(image_url)
            if response.status_code == 200:
                cols[col_idx].image(image_url, caption=f"Photo ID: {image_data['id']}")
                col_heights[col_idx] += 1
                displayed_images += 1
            else:
                st.warning(f"Failed to load image: {image_data['id']}")

if _name_ == "_main_":
    st.set_page_config(page_title="Flickr Image Search", page_icon="📷", layout="wide")
    
    st.title('🔍 Flickr Image Search')
    st.write('Search for photos by tags or keywords using Elasticsearch.')

    client = Elasticsearch("http://localhost:9200/")
    
    st.sidebar.title('Search Options')
    search_type = st.sidebar.selectbox('Choose Search Type:', ['match', 'fuzzy'])

    with st.form("search_form"):
        query = st.text_input('Enter search query:')
        submit = st.form_submit_button("Search")

    if submit:
        if query.strip():
            display_results(client, query, search_type)
        else:
            st.warning("Please enter a search query.")

In [ ]:
len(x)

In [1]:
ES_HOST = "http://127.0.0.1:9200/"
ES_USER = "elastic"
ES_PASSWORD = "changeme"
ES_TIMEOUT = 360000

DEST_INDEX = "my-image-embeddings"
DELETE_EXISTING = True
CHUNK_SIZE = 100

PATH_TO_IMAGES = "../app/static/images/**/*.jp*g"
PREFIX = "../app/static/images/"

In [5]:
from feature_extractor import FeatureExtractor
from elasticsearch import Elasticsearch
from PIL import Image
import os


def create_image_embeddings(es_host):
    es = Elasticsearch(
        ES_HOST,
        request_timeout=ES_TIMEOUT,
    )
    
    index_name = 'my-image-embeddings'
    
    # Ensure the index exists
    if not es.indices.exists(index=index_name):
        es.indices.create(index=index_name)
    
    # Set up your feature extractor
    feature_extractor = FeatureExtractor()  # Initialize your extractor

    images_directory = './static/images'
    for root, dirs, files in os.walk(images_directory):
        for file in files:
            if file.endswith(('jpeg', 'jpg', 'png')):
                image_path = os.path.join(root, file)
                image_id = os.path.splitext(file)[0]
                
                try:
                    # Generate embeddings
                    img = Image.open(image_path)  # Open the image file
                    embedding = feature_extractor.extract(img)
                    print(embedding)
                    
                    # Create document
                    doc = {
                        "image_id": image_id,
                        "image_name": file,
                        "image_embedding": embedding.tolist(),  # Convert numpy array to list
                        "relative_path": os.path.relpath(image_path, images_directory)
                    }
                    print(doc)
                    
                    # Save document to Elasticsearch
                    es.index(index=index_name, document=doc)
                except Exception as e:
                    print(f"Error processing {file}: {e}")

if __name__ == '__main__':
    try:
        create_image_embeddings(ES_HOST)
    except Exception as e:
        print(f"An error occurred while creating image embeddings: {e}")


[0.04424154 0.         0.         ... 0.01589872 0.         0.        ]
{'image_id': '5baebd41c0', 'image_name': '5baebd41c0.jpg', 'image_embedding': [0.04424154385924339, 0.0, 0.0, 0.0, 0.0, 0.002157551469281316, 0.04433829337358475, 0.0, 0.0, 0.00629708357155323, 0.020494380965828896, 0.0, 0.0, 0.04005035385489464, 0.0, 0.0, 0.01673971116542816, 0.0, 0.0, 0.0, 0.026397742331027985, 0.030109869316220284, 0.05490498244762421, 0.0, 0.008789523504674435, 0.0, 0.0, 0.0, 0.0, 0.0, 0.041165612637996674, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00991782359778881, 0.0, 0.0, 0.02675090916454792, 0.0, 0.0, 0.04064149782061577, 0.0, 0.0, 0.0, 0.0, 0.046527598053216934, 0.0, 0.0, 0.01333718840032816, 0.004409009590744972, 0.0, 0.0, 0.0, 0.01770290546119213, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0014291881816461682, 0.01839924417436123, 0.0, 0.02024177275598049, 0.0, 0.0085814343765378, 0.0, 0.0061878301203250885, 0.003017256036400795, 0.0, 0.0, 0.0, 0.0, 0.06534674018621445, 0

KeyboardInterrupt: 

In [ ]:
import os
import shutil

def copy_files_to_output(input_folder, output_folder):
    # Ensure the output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Traverse the directory tree from the input folder
    for subdir, _, files in os.walk(input_folder):
        for file in files:
            # Full path of the source file
            src_file_path = os.path.join(subdir, file)
            
            # Full path of the destination file
            dest_file_path = os.path.join(output_folder, file)
            
            # Copy the file to the output folder
            shutil.copy2(src_file_path, dest_file_path)
            print(f"Copied: {src_file_path} -> {dest_file_path}")

# Example usage:
input_folder = r'C:\Users\08you\Desktop\VGG16 project\animals\animals'
output_folder = r'C:\Users\08you\Desktop\VGG16 project\static\images'
copy_files_to_output(input_folder, output_folder)
